In [1]:
import numpy as np
import glob
import os
from skimage import morphology
#import sys
import scipy

#import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import cv2
#import imutils
#import pandas as pd
#import re
import datetime

import zipfile
from PIL import Image
Image.MAX_IMAGE_PIXELS = None

from sklearn import metrics
#from sklearn.utils import shuffle
#from skimage import draw
from skimage.io import imread

#from sklearn.model_selection import train_test_split#import spams

from skimage.util import pad
import skimage

import time
import math
import keras
from keras.models import model_from_json
#from keras.layers import *
#from keras.preprocessing import image
import keras.backend as K
#import random
#from sklearn.metrics import f1_score
#from skimage.transform import resize
from skimage.filters import threshold_otsu
#from tqdm import tqdm_notebook as tqdm 
#from tensorflow.keras.callbacks import TensorBoard

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction=0.29
config.gpu_options.allow_growth=True ##to use gpu as needed
config.log_device_placement = True  
sess = tf.Session(config=config)
set_session(sess)
K.tensorflow_backend._get_available_gpus()



 

Using TensorFlow backend.
W1014 18:26:11.931169 140518152927040 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1014 18:26:11.936409 140518152927040 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
a=np.zeros((8,512,512,1))
a=np.squeeze(a)#np.rollaxis(a,3,0)#[0]
print(a.shape)

(8, 512, 512)


In [3]:
!nvidia-smi

Mon Oct 14 18:26:12 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.39       Driver Version: 418.39       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:B1:00.0 Off |                    0 |
| N/A   34C    P0    53W / 300W |    318MiB / 32480MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [4]:
def dice(y_true, y_pred):
    
    y_true_f = (y_true).reshape((-1,1))
    y_pred_f = np.round((y_pred).reshape((-1,1)))
    

    intersection = np.nansum(y_true_f * y_pred_f)+(np.finfo(float).eps)
    union=np.nansum(y_true_f) + np.nansum(y_pred_f)
    
    dice= (2 * intersection) / ( union+ (np.finfo(float).eps))
    return dice 


def recall(y_true, y_pred):
    
    y_true_f = (y_true).reshape((-1,1))
    y_pred_f = np.round((y_pred).reshape((-1,1)))
    tp = np.nansum(y_true_f * y_pred_f)+(np.finfo(float).eps)
    fn=np.nansum((1-y_pred_f)*y_true_f)+(np.finfo(float).eps)
    recall= tp/(tp+fn+np.finfo(float).eps)
    return recall

# def auc(y_true, y_pred):
#     #y_true_f = K.flatten(y_true)
#     #y_pred_f = K.flatten(y_pred)
#     auc = tf.metrics.auc(y_true, y_pred,curve='PR',summation_method="careful_interpolation")[1]
#     K.get_session().run(tf.local_variables_initializer())
#     return auc

# def cm (y_true, y_pred):
    
#     auc = tf.metrics.auc(y_true, y_pred,curve='PR',summation_method="careful_interpolation")[1]
#     K.get_session().run(tf.local_variables_initializer())
    
#     y_true_f = K.flatten(y_true)
#     y_pred_f = K.flatten(y_pred)
    
#     intersection = K.sum(y_true_f * y_pred_f)
#     union=K.sum(y_true_f) + K.sum(y_pred_f)
    
#     dice= (2. * intersection) / ( union+ K.epsilon())
    
#     return (dice*100+auc*100)/2
    

In [5]:
dir_content=os.listdir(os.getcwd())
model_list=[s for s in dir_content if 'model_2019' in s]
model_list=sorted(model_list)
model_folder=os.path.join(os.getcwd(),model_list[-4])
print("available models are :",model_list,"\n","selected model is in :", model_folder)
model_weights=os.path.join(os.getcwd(),model_folder)+"/"+[weights for weights in os.listdir(model_folder) \
                                                          if 'model_' in weights][-1]
model_architecture=glob.glob(os.path.join(os.getcwd(),model_folder)+"/*.json")[-1]
print("Selected model file is : {}\nSelected architecture file is : {}".format(model_weights.split('/')[-1]\
                                                                          ,model_architecture.split('/')[-1]))


available models are : ['model_2019_08_16', 'model_2019_08_21', 'model_2019_08_23', 'model_2019_08_26', 'model_2019_08_27', 'model_2019_08_29', 'model_2019_08_30', 'model_2019_09_19', 'model_2019_09_21'] 
 selected model is in : /datalab/digest/Colonoscopy_tissue_segment_dataset/model_2019_08_29
Selected model file is : model_final_weights_2019_08_29.h5
Selected architecture file is : model_2019_08_29.json


In [6]:
json_file = open(model_architecture, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
OPT=keras.optimizers.SGD(lr=0.01,momentum=0.8,nesterov=True)
loaded_model.load_weights(model_weights)
loaded_model.compile(optimizer = OPT, loss = 'binary_crossentropy', metrics = ['accuracy'])
print(loaded_model.metrics_names)
loaded_model.summary()

W1014 18:26:13.850787 140518152927040 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1014 18:26:13.887876 140518152927040 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4185: The name tf.truncated_normal is deprecated. Please use tf.random.truncated_normal instead.

W1014 18:26:13.950279 140518152927040 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1014 18:26:14.017348 140518152927040 deprecation_wrapper.py:119] From /opt/conda/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:131: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1014 18:26:14.026925 140518152927040 depre

['loss', 'acc']
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_103 (Conv2D)             (None, 512, 512, 64) 1792        input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_104 (Conv2D)             (None, 512, 512, 64) 36928       conv2d_103[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_10 (MaxPooling2D) (None, 256, 256, 64) 0           conv2d_104[0][0]                 
_____________________________________________________________________________________________

from skimage import img_as_uint
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
def combined_metric(y_true, y_pred):
    
    y_true_f = (y_true).reshape((-1,1))
    y_pred_f = np.round((y_pred).reshape((-1,1)))
    
    
    p,r,_=metrics.precision_recall_curve(y_true_f, y_pred_f)
    auc=metrics.auc(p,r)
    
    intersection = np.nansum(y_true_f * y_pred_f)+(np.finfo(float).eps)
    union=np.nansum(y_true_f) + np.nansum(y_pred_f)
    
    dice= (2 * intersection) / ( union+ (np.finfo(float).eps))
    
    
    
    return (dice*100+auc*100)/2

def whole_img_pred(IMAGE_PATH,img_list,pred_dir_name,MASK_PATH=None,cal_performace=True):
    samples_per_step=5
    
    pred_dir=os.path.join(os.getcwd(),pred_dir_name)
    
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
        print("Made {} directory".format(pred_dir.split('/')[-1]))
    else:
        print("{} directory already exists in {}".format(pred_dir.split('/')[-1],'/'.join(pred_dir.split('/')[:-1])))
        
        

    avg_score=0
    test_num=len(img_list)
    for img_name in [img_list[1]]:
        print(img_name)
        str_time=time.time()

        img=imread(os.path.join(IMAGE_PATH,img_name))
        r,c,_=img.shape#4663,3881
      
        new_r_count=(math.ceil((r-512)/512)+1)#10
        new_c_count=(math.ceil((c-512)/512)+1)#8


        pad_r1=((new_r_count-1)*512-r+512)//2 #228
        pad_r2=((new_r_count-1)*512-r+512)-pad_r1 #229
        pad_c1=((new_c_count-1)*512-c+512)//2 #107
        pad_c2=((new_c_count-1)*512-c+512)-pad_c1#108

        arr_in=np.pad(img, [(pad_r1,pad_r2),(pad_c1,pad_c2),(0,0)], 'constant', constant_values=0)/255
        print(arr_in.shape)
        
        window_shape=(512,512,3)
        arr_out=skimage.util.view_as_windows(arr_in, window_shape, step=512)
        x,y=arr_out.shape[:2]
        print(x,y)
        ar2=arr_out.reshape((-1,512,512,3))
        del arr_out,arr_in,img
        y_count=0
        img_temp=[]
        #y_pred=np.empty()
        check=True
        i_prev=0
        y_iter=y//samples_per_step
        y_iter_counter=1
        y_dif=y-y_iter
        for i in range(math.ceil(x*y/samples_per_step)):
            if (i)*samples_per_step<y_iter:
                if check:
                    y_pred=np.concatenate(loaded_model.predict(ar2[i*samples_per_step:\
                                                                   (i+1)*samples_per_step]),axis=1)
                    check=False
                    print(y_pred.shape,"Check1",i)
                else:
                    y_pred=np.append(y_pred,np.concatenate(loaded_model.predict(ar2[i*samples_per_step:(i+1)*\
                                                                                    samples_per_step]),axis=1),axis=1)
                    print(y_pred.shape,"Check2",i)
            else:
                y_pred=np.append(y_pred,np.concatenate(loaded_model.predict(ar2[(i)*samples_per_step:\
                                                                                (i)*samples_per_step+y_dif\
                                                                           ]),axis=1),axis=1)
                print("row {} :{}".format(i,y_pred.shape))
                img_temp.append(y_pred)
                check=True
                del y_pred
                y_iter_counter+=1
                y_iter*=y_iter_counter
                
        img_temp=np.array(img_temp,dtype=np.float_)
        print(img_temp.shape,'Check4')
        img_temp=np.concatenate(img_temp,axis=0)
        print(img_temp.shape,"Check5")
        img_temp=img_temp[pad_r1:img_temp.shape[0]-pad_r2,pad_c1:img_temp.shape[1]-pad_c2,:]*255
        print(img_temp.shape,"Check6")
        #thresh = threshold_otsu(img_temp)
        #img_temp = np.array(img_temp > thresh,dtype=np.float32)
        pred_img_name=img_name.split('.')[0]+'_pred.jpg'
        pred_img_path=os.path.join(pred_dir,pred_img_name)
        skimage.io.imsave(pred_img_path,img_as_uint(img_temp))
        print("time for {} shaped {} is {} s".format((r,c),img_name,time.time()-str_time))
        if cal_performace:
            mask_path=os.path.join(MASK_PATH,img_name.split('.')[0]+'_mask.jpg')
            mask=imread(mask_path,as_gray=True)//255
            score=combined_metric(mask, img_temp)
            avg_score+=score
            print("Score : {}".format(score))
        del img_temp,ar2
        if cal_performace:
            del mask
    if cal_performace:
        print("Average Score : {}".format(avg_score/test_num))
    
    print("DONE")
        
MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_color_normalized'

#NEG_IMAGE_PATH='/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-neg'
#neg_img_list=os.listdir(NEG_IMAGE_PATH)

all_img_list=os.listdir(IMAGE_PATH)
INPUT_IMAGE_PATH_TEST='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input_cn_overlap_test'


test_files=os.listdir(INPUT_IMAGE_PATH_TEST)
test_img_list=['_'.join(x.split('_')[:-2])+'.jpg' for x in test_files]


#whole_img_pred(IMAGE_PATH,test_img_list,'whole_pred_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d")),\
#               MASK_PATH,cal_performace=True)


        
        
        
        

from skimage import img_as_uint
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
def combined_metric(y_true, y_pred):
    
    y_true_f = (y_true).reshape((-1,1))
    y_pred_f = np.round((y_pred).reshape((-1,1)))
    
    
    p,r,_=metrics.precision_recall_curve(y_true_f, y_pred_f)
    auc=metrics.auc(p,r)
    
    intersection = np.nansum(y_true_f * y_pred_f)+(np.finfo(float).eps)
    union=np.nansum(y_true_f) + np.nansum(y_pred_f)
    
    dice= (2 * intersection) / ( union+ (np.finfo(float).eps))
    
    
    
    return (dice*100+auc*100)/2

def whole_img_pred(IMAGE_PATH,img_list,pred_dir_name,MASK_PATH=None,cal_performace=True):
    samples_per_step=5
    
    pred_dir=os.path.join(os.getcwd(),pred_dir_name)
    
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
        print("Made {} directory".format(pred_dir.split('/')[-1]))
    else:
        print("{} directory already exists in {}".format(pred_dir.split('/')[-1],'/'.join(pred_dir.split('/')[:-1])))
        
        

    avg_score=0
    test_num=len(img_list)
    for img_name in [img_list[1]]:
        print(img_name)
        str_time=time.time()

        img=imread(os.path.join(IMAGE_PATH,img_name))
        r,c,_=img.shape#4663,3881
      
        new_r_count=(math.ceil((r-512)/512)+1)#10
        new_c_count=(math.ceil((c-512)/512)+1)#8


        pad_r1=((new_r_count-1)*512-r+512)//2 #228
        pad_r2=((new_r_count-1)*512-r+512)-pad_r1 #229
        pad_c1=((new_c_count-1)*512-c+512)//2 #107
        pad_c2=((new_c_count-1)*512-c+512)-pad_c1#108

        arr_in=np.pad(img, [(pad_r1,pad_r2),(pad_c1,pad_c2),(0,0)], 'constant', constant_values=0)/255
        print(arr_in.shape)
        
        window_shape=(512,512,3)
        arr_out=skimage.util.view_as_windows(arr_in, window_shape, step=512)
        x,y=arr_out.shape[:2]
        print(x,y)
        ar2=arr_out.reshape((-1,512,512,3))
        del arr_out,arr_in,img
        y_count=0
        img_temp=[]
        #y_pred=np.empty()
        check=True
        i_prev=0
        y_iter=y//samples_per_step
        y_iter_counter=1
        y_dif=y-y_iter
        for i in range(x):
            for j in range(y):
                if j<y_iter
                    if j==0:
                        y_pred=np.concatenate(loaded_model.predict(ar2[i*samples_per_step:\
                                                                       (i+1)*samples_per_step]),axis=1)
                        check=False
                        print(y_pred.shape,"Check1",i)
                    else:
                        y_pred=np.append(y_pred,np.concatenate(loaded_model.predict(ar2[i*samples_per_step:(i+1)*\
                                                                                        samples_per_step]),axis=1),axis=1)
                        print(y_pred.shape,"Check2",i)
                else:
                    y_pred=np.append(y_pred,np.concatenate(loaded_model.predict(ar2[(i)*samples_per_step:\
                                                                                    (i)*samples_per_step+y_dif\
                                                                               ]),axis=1),axis=1)
                    print("row {} :{}".format(i,y_pred.shape))
                    img_temp.append(y_pred)
                    check=True
                    del y_pred
                    y_iter_counter+=1
                    y_iter*=y_iter_counter
                
        img_temp=np.array(img_temp,dtype=np.float_)
        print(img_temp.shape,'Check4')
        img_temp=np.concatenate(img_temp,axis=0)
        print(img_temp.shape,"Check5")
        img_temp=img_temp[pad_r1:img_temp.shape[0]-pad_r2,pad_c1:img_temp.shape[1]-pad_c2,:]*255
        print(img_temp.shape,"Check6")
        thresh = threshold_otsu(img_temp)
        img_temp = np.array(img_temp > thresh,dtype=np.float32)
        pred_img_name=img_name.split('.')[0]+'_pred.jpg'
        pred_img_path=os.path.join(pred_dir,pred_img_name)
        skimage.io.imsave(pred_img_path,img_as_uint(img_temp))
        print("time for {} shaped {} is {} s".format((r,c),img_name,time.time()-str_time))
        if cal_performace:
            mask_path=os.path.join(MASK_PATH,img_name.split('.')[0]+'_mask.jpg')
            mask=imread(mask_path,as_gray=True)//255
            score=combined_metric(mask, img_temp)
            avg_score+=score
            print("Score : {}".format(score))
        del img_temp,ar2
        if cal_performace:
            del mask
    if cal_performace:
        print("Average Score : {}".format(avg_score/test_num))
    
    print("DONE")
        
MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_color_normalized'

#NEG_IMAGE_PATH='/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-neg'
#neg_img_list=os.listdir(NEG_IMAGE_PATH)

all_img_list=os.listdir(IMAGE_PATH)
INPUT_IMAGE_PATH_TEST='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input_cn_overlap_test'


test_files=os.listdir(INPUT_IMAGE_PATH_TEST)
test_img_list=['_'.join(x.split('_')[:-2])+'.jpg' for x in test_files]


#whole_img_pred(IMAGE_PATH,test_img_list,'whole_pred_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d")),\
              # MASK_PATH,cal_performace=True)


        
        
        
        

In [7]:
def snnmf(I):
    
    I_lab_space = cv2.cvtColor(I, cv2.COLOR_RGB2LAB)
    luminosity = I_lab_space[:, :, 0] 
    mask_lumin = luminosity/ 255 < 0.8
    
    mask_zero = (I == 0)
    I[mask_zero] = 1
    
    od = (np.maximum(-1 * np.log(I / 255), 1e-6))
    od_dl = od[mask_lumin]
    od=od.reshape((-1, 3))
    od_dl=od_dl.reshape((-1, 3))

    dictionary = spams.trainDL(X=od_dl.T, K=2, lambda1=0.1, mode=2,
                               modeD=0, posAlpha=True, posD=True, verbose=False).T

    # dictionary is 2 x 3
    #First one should be Hematoxylin(Bluish-purple)
    if dictionary[0, 0] < dictionary[1, 0]:
        dictionary = dictionary[[1, 0], :]

    dictionary=dictionary / np.linalg.norm(dictionary, axis=1)[:, None]
    
    sparse=spams.lasso(X=od.T, D=dictionary.T, mode=2, lambda1=0.01, pos=True).toarray().T
    
    return dictionary,sparse

def h_norm(hs,ht):
    
    ht_rm=np.percentile(ht, 99, axis=0).reshape(1,2)
    hs_rm=np.percentile(hs, 99, axis=0).reshape(1,2)
    
    hs_norm_mat=hs*ht_rm/hs_rm
    return hs_norm_mat

In [8]:
from skimage import img_as_uint
import random
import csv
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

def whole_img_pred(IMAGE_PATH,img_list,pred_dir_name,MASK_PATH=None,cal_performace=True):
    samples_per_step=5
    threshold=0.8
    
    pred_dir=os.path.join(os.getcwd(),pred_dir_name)
    
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
        print("Made {} directory".format(pred_dir.split('/')[-1]))
    else:
        print("{} directory already exists in {}".format(pred_dir.split('/')[-1],'/'.join(pred_dir.split('/')[:-1])))
        
    #imgt=imread(os.path.join(TARGET_PATH,'18-00530B_2019-05-07 23_56_22-lv1-11712-16122-7372-7686.jpg'))    
    #wt,ht=snnmf(imgt)
    avg_score=0
    test_num=len(img_list)
    random.shuffle(img_list)
    iterations=2
    lines=[]
    for img_name in img_list:
        #print(img_name)
        str_time=time.time()

        #imgs=imread(os.path.join(IMAGE_PATH,img_name))
        img=imread(os.path.join(IMAGE_PATH,img_name))
        #ws,hs=snnmf(imgs)
        
        #hs_norm=h_norm(hs,ht)
        #od_recon=255 * np.exp(-1 * np.dot(hs_norm, wt))
        #img=od_recon.reshape(imgs.shape).astype(np.uint8)
        
        r,c,_=img.shape#4663,3881
      
        new_r_count=(math.ceil((r-512)/512)+1)#10
        new_c_count=(math.ceil((c-512)/512)+1)#8


        pad_r1=((new_r_count-1)*512-r+512)//2 #228
        
        pad_r2=((new_r_count-1)*512-r+512)-pad_r1 #229
        pad_c1=((new_c_count-1)*512-c+512)//2 #107
        pad_c2=((new_c_count-1)*512-c+512)-pad_c1#108
        

        arr_in=np.pad(img, [(pad_r1,pad_r2),(pad_c1,pad_c2),(0,0)], 'constant', constant_values=0)/255
        
        
        window_shape=(512,512,3)
        arr_out=skimage.util.view_as_windows(arr_in, window_shape, step=512)
        x,y=arr_out.shape[:2]
        ar2=arr_out.reshape((-1,512,512,3))
        del arr_out,arr_in,img
        img_temp=[]
        check=True
        y_iter=y//samples_per_step
        y_dif=y%5
        i=0
        i_temp=0
        count=0
        while i <=(x*y):
            
            if i_temp<y_iter or y_iter==0:                
                if i_temp==0:
                    y_pred=np.concatenate(loaded_model.predict(ar2[i:i+samples_per_step]),axis=1)
                else:
                    y_pred=np.append(y_pred,np.concatenate(loaded_model.predict(ar2[i:i+samples_per_step]),axis=1),axis=1)
                i+=5
                i_temp+=1
                
                    
            else:
                count+=1
            
                if count==x:
                    break
                    
                if y_dif!=0:
                    y_pred=np.append(y_pred,np.concatenate(loaded_model.predict(ar2[i:i+y_dif]),axis=1),axis=1)
                    i+=y_dif
                img_temp.append(y_pred)
        
                
                i_temp=0
                del y_pred                    
                
        img_temp=np.array(img_temp,dtype=np.float_)
        img_temp=np.concatenate(img_temp,axis=0)
        img_temp=np.squeeze(img_temp[pad_r1:img_temp.shape[0]-pad_r2,pad_c1:img_temp.shape[1]-pad_c2,:],axis=2)#*255
    
        img_prob=img_temp[img_temp>=0.4]
        img_pos_pixels=img_prob[img_prob>=threshold]
        img_neg_pixels=img_prob[img_prob<threshold]
        prob_temp=len(img_pos_pixels)/len(img_prob)
        if prob_temp>0.4:
            prob_score=np.max(img_pos_pixels)
        else:
            prob_score=np.min(img_neg_pixels)
            
        #print(prob_score," prob")
        
        #(((np.mean(img_pos_pixels)/len(img_pos_pixels)))+((np.mean(img_neg_pixels)/len(img_neg_pixels))))*len(img_prob)
        #(np.sum(img_pos_pixels)+np.sum(img_neg_pixels))/len(img_prob)#np.sum(img_neg_pixels)
        
        #predict.csv
       
        img_temp_thresh = np.array(img_temp > threshold,dtype=np.float32)        
        #img_temp=scipy.ndimage.morphology.binary_fill_holes(img_temp_thresh)
        #img_temp = morphology.remove_small_objects(img_temp, 2000).astype(np.float)    
        #img_temp_seed=cv2.erode(img_temp, kernel, iterations=1)
        #img_temp=skimage.morphology.reconstruction(img_temp_seed,img_temp_thresh,selem=skimage.morphology.selem.disk(3))
        #img_temp=(scipy.ndimage.morphology.binary_fill_holes(img_temp)).astype(np.float)
        
        #img_temp=(scipy.ndimage.morphology.binary_fill_holes(img_temp)).astype(np.float)        
        pred_img_name=img_name#.split('.')[0]+'_pred.jpg'
        lines.append([pred_img_name,prob_score])
        pred_img_path=os.path.join(pred_dir,pred_img_name)
        skimage.io.imsave(pred_img_path,img_as_uint(img_temp))
        #print("time for {} shaped {} is {} s".format((r,c),img_name,time.time()-str_time))
        del img_temp,ar2
        with open('predictions.csv', mode='w') as writeFile:
            fields=['image_name','score']
            writer = csv.writer(writeFile)
            writer.writerow(fields)
            writer.writerows(lines)
    
    print("DONE")

    
#### PREDICT ###
#pred_dir_name='/input'
#img_list=os.listdir(pred_dir_name)

#output_dir_name='/output'

######
MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_color_normalized'
NEG_IMAGE_PATH='/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-neg_norm'

#NEG_IMAGE_PATH='/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-neg'
neg_img_list=os.listdir(NEG_IMAGE_PATH)

all_img_list=os.listdir(IMAGE_PATH)
INPUT_IMAGE_PATH_TEST='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input_cn_overlap_test'


test_files=os.listdir(INPUT_IMAGE_PATH_TEST)
test_img_list=['_'.join(x.split('_')[:-2])+'.jpg' for x in test_files]
test_img_list=list(dict.fromkeys(test_img_list))

# whole_img_pred(IMAGE_PATH,test_img_list,'whole_pred_junk_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d")),\
#                MASK_PATH,cal_performace=True)


        
        
        
        

In [ ]:
from skimage import img_as_uint
def whole_img_pred(IMAGE_PATH,img_list,pred_dir_name,MASK_PATH=None,cal_performace=True):
    
    pred_dir=os.path.join(os.getcwd(),pred_dir_name)
    
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
        print("Made {} directory".format(pred_dir.split('/')[-1]))
    else:
        print("{} directory already exists in {}".format(pred_dir.split('/')[-1],'/'.join(pred_dir.split('/')[:-1])))
        
        

    avg_recall=0
    avg_dice=0
    test_num=len(img_list)
    left_scale=np.ones((512,512))
    left_scale[:256,:]=0.5
    right_scale=np.ones((512,512))
    right_scale[256:,:]=0.5
    kernel=skimage.morphology.disk(2, dtype=np.uint8)
    for img_name in img_list:
        str_time=time.time()

        img=imread(os.path.join(IMAGE_PATH,img_name))
        r,c,_=img.shape#4663,3881
        print("Original image shape is {}".format((r,c)))
      
        new_r_count=(math.ceil((r-512)/512)+1)#10
        new_c_count=(math.ceil((c-512)/256)+1)#8


        pad_r1=((new_r_count-1)*512-r+512)//2 #228
        pad_r2=((new_r_count-1)*512-r+512)-pad_r1 #229
        pad_c1=((new_c_count-1)*512-c+512)//2 #107
        pad_c2=((new_c_count-1)*512-c+512)-pad_c1#108

        arr_in=np.pad(img, [(pad_r1,pad_r2),(pad_c1,pad_c2),(0,0)], 'constant', constant_values=0)/255
        print("Padded image dimensions are {}".format(arr_in.shape))
        window_shape=(512,512,3)
        arr_out=skimage.util.view_as_windows(arr_in, window_shape, step=512)
        x,y=arr_out.shape[:2]
        print(" Number of rows are {} and columns are {}".format(x,y))
        ar2=arr_out.reshape((-1,512,512,3))
        del arr_out,arr_in,img
        y_pred=loaded_model.predict(ar2)
        
    
        img_temp=[]
        for i in range(x):
            img_temp.append(np.concatenate(y_pred[i*y:(i+1)*y],axis=1))
        img_temp=np.array(img_temp,dtype=np.float_)

        img_temp=np.concatenate(img_temp,axis=0)
           
#         y_pred=np.squeeze(loaded_model.predict(ar2))
        
        
    
#         img_temp=np.zeros((arr_in.shape[:2]))
#         for i in range(x):
#             print(i+1," ROW")
#             y_pred_temp=np.zeros((512,arr_in.shape[1]))
#             for j in range(y):
#                 y_pred_temp[:,j*256:j*256+512]+=y_pred[j]
                
#             img_temp[i*256:i*256+512,:]+=y_pred_temp
#             #img_temp.append(np.concatenate(y_pred_temp,axis=1))
#         img_temp=np.array(img_temp,dtype=np.float_)

#         img_temp=np.concatenate(img_temp,axis=0)
        img_temp=img_temp[pad_r1:img_temp.shape[0]-pad_r2,pad_c1:img_temp.shape[1]-pad_c2,:]#*255
        thresh = 0.8#threshold_otsu(img_temp)
        img_thresh_initial = np.squeeze(np.array(img_temp > 0.6,dtype=np.float32))
        print(img_thresh_initial.shape)
        img_thresh=scipy.ndimage.morphology.binary_fill_holes(img_thresh_initial)
        img_thresh = morphology.remove_small_objects(img_thresh, 2000).astype(np.float) 
        
        img_temp_seed=cv2.erode(np.float32(img_thresh), kernel, iterations=5)
        print(img_temp_seed.shape)
        img_thresh=skimage.morphology.reconstruction(img_temp_seed, img_thresh, method='dilation', selem=None, offset=None)
        pred_img_name=img_name.split('.')[0]+'_pred.jpg'
        pred_img_path=os.path.join(pred_dir,pred_img_name)
        skimage.io.imsave(pred_img_path,img_thresh)
        print("time for {} shaped {} is {} s".format((r,c),img_name,time.time()-str_time))
        if cal_performace:
            mask_path=os.path.join(MASK_PATH,img_name.split('.')[0]+'_mask.jpg')
            mask=imread(mask_path,as_gray=True)//255
            dice_score=dice(mask, img_thresh)
            recall_score=recall(mask,img_thresh)
            avg_dice+=dice_score
            avg_recall+=recall_score
            print("Score : {}\nRecall: {}".format(dice_score,recall_score))
        del img_temp,ar2
        if cal_performace:
            del mask
    if cal_performace:
        print("Average Dice : {}".format(avg_dice/test_num))
        print("Average Recall : {}".format(avg_recall/test_num))
    
    print("DONE")
        
MASK_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_mask'
IMAGE_PATH='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-pos-v1_color_normalized'
NEG_IMAGE_PATH='/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-neg_norm'

#NEG_IMAGE_PATH='/datalab/digest/Colonoscopy_tissue_segment_dataset/tissue-train-neg'
neg_img_list=os.listdir(NEG_IMAGE_PATH)

all_img_list=os.listdir(IMAGE_PATH)
INPUT_IMAGE_PATH_TEST='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/input_cn_overlap_test'


test_files=os.listdir(INPUT_IMAGE_PATH_TEST)
test_img_list=['_'.join(x.split('_')[:-2])+'.jpg' for x in test_files]
test_img_list=list(dict.fromkeys(test_img_list))

whole_img_pred(IMAGE_PATH,test_img_list,'entire_image_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d")),\
               MASK_PATH,cal_performace=True)


        
        
        
        

entire_image_2019_10_14 directory already exists in /datalab/digest/Colonoscopy_tissue_segment_dataset
Original image shape is (9314, 12688)
Padded image dimensions are (9728, 25088, 3)
 Number of rows are 19 and columns are 49
(9314, 12688)
(9314, 12688)


W1014 18:28:38.333432 140518152927040 util.py:64] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


time for (9314, 12688) shaped D201707788_2019-05-14 14_05_34-lv1-2777-15975-12688-9314.jpg is 124.58131122589111 s
Score : 0.6624420331597789
Recall: 0.8274544282336693
Original image shape is (28621, 19192)
Padded image dimensions are (28672, 37888, 3)
 Number of rows are 56 and columns are 74
(28621, 19192)
(28621, 19192)


## kernel=skimage.morphology.disk(2, dtype=np.uint8)
print(kernel)

In [ ]:
from zipfile import ZipFile
file_paths=[os.path.join('whole_pred_junk_2019_09_27',x) for x in os.listdir('whole_pred_junk_2019_09_27')]
with ZipFile('pred.zip','w') as zip: 
        # writing each file one by one 
        for file in file_paths: 
            zip.write(file) 

In [ ]:
path='/home/vahadaneabhi01/datalab/digest/Colonoscopy_tissue_segment_dataset/whole_pred_junk_2019_09_26/D20170502603_2019-05-14 11_39_00-lv1-19545-91-19192-28621.jpg'
img_temp=imread(path)

#kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(15,15))
img_temp2=cv2.erode(img_temp, skimage.morphology.selem.disk(11), iterations=1)
#img_temp=cv2.dilate(img_temp, kernel2, iterations=3)
#img_temp=cv2.erode(img_temp, kernel, iterations=2)
#img_temp=cv2.dilate(img_temp, kernel2, iterations=3)
plt.imshow(img_temp2)






In [ ]:
#img_tempx=cv2.erode(img_temp2, kernel, iterations=3)
img_temp3=skimage.morphology.reconstruction(img_temp2,img_temp,selem=skimage.morphology.selem.disk(3))
#img_temp=cv2.dilate(img_temp, kernel2, iterations=3)
#img_temp=cv2.erode(img_temp, kernel, iterations=2)
#img_temp=cv2.dilate(img_temp, kernel2, iterations=3)
plt.imshow(img_temp3)

In [ ]:
plt.imshow(img_temp2)


In [ ]:
img_temp4=(scipy.ndimage.morphology.binary_fill_holes(img_temp3)).astype(np.float)

In [ ]:
plt.imshow(img_temp4)

In [ ]:
skimage.io.imsave('asd2.jpg',img_temp4)

In [ ]:
print(skimage.morphology.selem.disk(3))

In [ ]:
plt.imshow(img_temp)

In [ ]:
print(kernel.flatten())

In [ ]:
a=cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(20,20))
print(type(a))

In [ ]:
whole_img_pred(NEG_IMAGE_PATH,neg_img_list,'whole_pred_neg_{}'.format(datetime.datetime.now().strftime("%Y_%m_%d")),\
               MASK_PATH,cal_performace=False)

In [ ]:
img='/datalab/digest/Colonoscopy_tissue_segment_dataset/2019-10962-1-1-1_2019-05-28 15_22_19-lv1-35695-22595-1269-1099.jpg'
img2=imread(img)
print(img2.shape)

In [ ]:
img=imread('/datalab/digest/Colonoscopy_tissue_segment_dataset/whole_pred_test_2019_09_25/D20170502603_2019-05-14 11_39_00-lv1-19545-91-19192-28621_pred.jpg')
kernel2 = np.ones((int(img.shape[0]*0.01),int(img.shape[1]*0.1)), np.uint8)
kernel=np.ones((25,25), np.uint8)
iterations=5

img=scipy.ndimage.morphology.binary_fill_holes(img)

img=cv2.dilate(img, kernel, iterations=iterations) 
img=cv2.erode(img, kernel, iterations=1)

#import scipy
#img2=scipy.ndimage.morphology.binary_fill_holes(img)
plt.imshow(img)


In [ ]:
test_img_list=list(dict.fromkeys(test_img_list))
print(len(test_img_list))

In [ ]:

img3 = morphology.remove_small_objects(img2, 150*150)
plt.imshow(img3)


In [ ]:
skimage.io.imsave('sample.jpg',img_as_uint(img))

In [ ]:
b=np.zeros((5,5,3))
b[2:4]=1
a=b[b!=0]
print(np.sum(a)/len(a),b)